# IMDB Reviews - Classification

In [44]:
import pandas as pd 
import tarfile
import os
import numpy as np 
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
np.set_printoptions(precision=2)

Raw data is seperated as train and test data. The positive and negative sentiments are again classified under two folders.

In [14]:
raw_data_file = r"C:\Users\sanjiv\Documents\DataScience\Projects\SentimentClassification\data\raw\aclImdb_v1.tar.gz"

In [15]:
def extract_tarfile(file_name):
    tar = tarfile.open(file_name, "r:gz")
    tar.extractall()
    tar.close()

In [16]:
#extract_tarfile(raw_data_file)

In [17]:
url_csv = 'https://raw.githubusercontent.com/TheClub4/IMDB_Sentiment_Analysis/master/movie_data.csv'

In [18]:
chunk_size=5000
batch_no=1
for chunk in pd.read_csv(url_csv,chunksize=chunk_size):
    #print(batch_no)
    chunk.to_csv('chunk'+str(batch_no)+'.csv',index=False)
    batch_no+=1

1
2
3
4
5
6
7
8
9
10


In [2]:
df = pd.DataFrame()#pd.read_csv(url_csv)

In [3]:
data_loc = r'C:\Users\sanjiv\Documents\DataScience\Projects\SentimentClassification\notebooks'

def find_csv_filenames( path_to_dir, suffix=".csv" ):
    filenames = os.listdir(data_loc)
    return [ filename for filename in filenames if filename.endswith(suffix )]


for csv_file in find_csv_filenames(data_loc):
    df = df.append(pd.read_csv(os.path.join(data_loc,csv_file), skiprows=1, header=None))


In [4]:
df.columns = ['review','sentiment']

In [8]:
df.reset_index(drop=True)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0
3,hi for all the people who have seen this wonde...,1
4,"I recently bought the DVD, forgetting just how...",0
...,...,...
49995,I had the privilege to watch Mar Adentro last ...,1
49996,Peter Cushing and Donald Pleasance are legenda...,0
49997,"I mean, come on! This movie had such nice pote...",0
49998,I'm sure this is a show no one is that familia...,1


In [5]:
df['sentiment'].value_counts()

0    25000
1    25000
Name: sentiment, dtype: int64

In [9]:
df['review']

0       In 1974, the teenager Martha Moxley (Maggie Gr...
1       OK... so... I really like Kris Kristofferson a...
2       ***SPOILER*** Do not read this, if you think a...
3       hi for all the people who have seen this wonde...
4       I recently bought the DVD, forgetting just how...
                              ...                        
4995    I had the privilege to watch Mar Adentro last ...
4996    Peter Cushing and Donald Pleasance are legenda...
4997    I mean, come on! This movie had such nice pote...
4998    I'm sure this is a show no one is that familia...
4999    It was a good story, but not very well told. I...
Name: review, Length: 50000, dtype: object

1. Remove punctuations

In [18]:
def remove_punctuation(string):
  
    # punctuation marks
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
  
    # traverse the given string and if any punctuation
    # marks occur replace it with null
    for x in string.lower():
        if x in punctuations:
            
            string = string.replace(x, "")
  
    # Return string without punctuation and 
    # change to lower case
    return string.lower()

In [19]:
df['review'] =df['review'].apply(lambda x:remove_punctuation(x))

In [20]:
df.head()


,review,sentiment
0,in 1974 the teenager martha moxley maggie grac...,1
1,ok so i really like kris kristofferson and his...,0
2,spoiler do not read this if you think about wa...,0
3,hi for all the people who have seen this wonde...,1
4,i recently bought the dvd forgetting just how ...,0


In [37]:
docs = [sentence for sentence in df['review']]

In [38]:
#print(docs)

count_vectorizer = CountVectorizer()

In [39]:
bag = count_vectorizer.fit_transform(docs)

In [43]:
#count_vectorizer.vocabulary_
bag.toarray()[0]

MemoryError: Unable to allocate 67.2 GiB for an array with shape (50000, 180328) and data type int64

In [45]:
tfidf = TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True)


In [72]:
tf = tfidf.fit_transform(bag)

In [73]:
tf.shape

(50000, 180328)

In [50]:
np.argmin(tfidf.idf_)

159321

AttributeError: 'TfidfTransformer' object has no attribute 'shape'

In [56]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

C:\Users\sanjiv\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
C:\Users\sanjiv\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=Tru

In [57]:
train_bow = bag[:40000,:]
test_bow = bag[40000:,:]

In [59]:
train_bow.shape

(40000, 180328)

In [61]:
xtrain_bow, xvalid_bow, ytrain, yvalid = train_test_split(train_bow, df['sentiment'][:40000], random_state=42, test_size=0.3)

In [64]:
ytrain.shape

(28000,)

In [65]:
lreg = LogisticRegression()

In [66]:
lreg.fit(xtrain_bow, ytrain)

C:\Users\sanjiv\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [67]:
pred = lreg.predict_proba(xvalid_bow)

In [69]:
yval_pred = lreg.predict(xvalid_bow)

C:\Users\sanjiv\Anaconda3\lib\site-packages\sklearn\linear_model\base.py:291: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  indices = (scores > 0).astype(np.int)


In [70]:
f1_score(yvalid, yval_pred)

0.8894754206532496

In [74]:
train_tfidf = tf[:40000,:]
test_tfidf= tf[40000:,:]

In [76]:
xtrain_tf, xvalid_tf, ytrain, yvalid = train_test_split(train_tfidf, df['sentiment'][:40000], random_state=42, test_size=0.3)

In [77]:
lreg_tf = LogisticRegression()
lreg_tf.fit(xtrain_tf,ytrain)

C:\Users\sanjiv\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [78]:
ypred_tf = lreg_tf.predict(xvalid_tf)

C:\Users\sanjiv\Anaconda3\lib\site-packages\sklearn\linear_model\base.py:291: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  indices = (scores > 0).astype(np.int)


In [80]:
f1_score(yvalid, ypred_tf)

0.8919317809117745

In [86]:
pred = lreg_tf.predict(test_tfidf[1,:])

C:\Users\sanjiv\Anaconda3\lib\site-packages\sklearn\linear_model\base.py:291: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  indices = (scores > 0).astype(np.int)


In [89]:
test_tfidf[:1,:]

<1x180328 sparse matrix of type '<class 'numpy.float64'>'
	with 130 stored elements in Compressed Sparse Row format>

In [90]:
pred

array([1], dtype=int64)